# Combine Video & Object

## Dependencies
- Python ≥3.6 on Linux or Python ≥3.8 on Windows (which avoids WinError 6, see https://git.io/Jv8cN)
- moviepy
  - If you run into an error: `AttributeError: 'NoneType' object has no attribute 'stdout'` (see https://git.io/JvlyN) on moviepy version 1.0.1 try downgrading to version 1.0.0 of moviepy, explictly: `pip install moviepy==1.0.0`

## Usage
### Folder Structure

The script is expecting the following folder structure:
 
📂Video_Folder1  
├─ Video_File_1.avi  
├─ Video_File_2.avi  
├─ Video_File_n.avi  
📂Video_Folder2  
├─ Video_File_1.avi  
├─ Video_File_2.avi  
├─ Video_File_n.avi    
📂Object_Folder1  
├─ Object_File_1.png  
├─ Object_File_2.png  
├─ Object_File_n.png  
📂Object_Folder2  
├─ Object_File_1.png  
├─ Object_File_2.png  
├─ Object_File_n.png  
 
### Merging Strategy
In each iteration a video file will be combined with every image file. For example, `Video-File-1.avi` will be combined with all image files within the object folders, successively. The folder and file names will be combined, too. The resulting video files will be stored in an `/out` folder. When running only the first video from folder list above, the following structure would appear:

`script.ipynb`  
📂\[other folders from above\]  
📂out  
├─Video_Folder1-Video_File_1-Object_Folder1-Object_File_1.avi  
├─Video_Folder1-Video_File_1-Object_Folder1-Object_File_2.avi  
├─Video_Folder1-Video_File_1-Object_Folder1-Object_File_n.avi  
├─Video_Folder1-Video_File_1-Object_Folder2-Object_File_1.avi  
├─Video_Folder1-Video_File_1-Object_Folder2-Object_File_2.avi  
├─Video_Folder1-Video_File_1-Object_Folder2-Object_File_n.avi  

### ✏️ Edit and Adjust the Script
Sections containing a ✏️ icon should be adjusted to meet specific requirements.

## ▶ Generate Video Script
After installing `moviepy` (see above), start executing the script:

In [ ]:
import time
from pathlib import Path # pathlib > os.path.join (see http://tiny.cc/d1o1jz)
import moviepy.editor as mp

### ✏️ Folders
Define video and object folders.

In [ ]:
video_folders = ("Inter_A", "Inter_B")
object_folders = ("ObjectW_a", "ObjectW_b", "ObjectX_a", "ObjectX_b", "ObjectY_a", "ObjectY_b", "ObjectZ_a", "ObjectZ_b")

### ℹ File & Folder Information

In [ ]:
total_video_files = 0
total_object_files = 0

print("-------------------")
print("Video Files Count:")
for cv in video_folders:
    total_video_files += len(list(Path(cv).glob("**/*")))
    print(f"{cv}:\t{len(list(Path(cv).glob('**/*')))}")

print("-------------------")
print("Object Files Count:")
for co in object_folders:
    total_object_files += len(list(Path(co).glob("**/*")))
    print(f"{co}:\t{len(list(Path(co).glob('**/*')))}")
print("-------------------")

print(f"File Info:\nMerging all video files with all object files will result in {total_video_files * total_object_files} video files.")

### ✏️ Parameters
Here you can adjust the *delitermer character* to be used for separating folder names and file names (default is `"-"`), the *pixel coordinates* for inserting the object, and other stuff.

#### About Coordinates
The top left pixel of the object is the coordinates origin in moviepy. For example, if the video has height of 1152px and the object has height of 260px, and you want to place the video at the bottom of the screen you can define `object_y_pos = 1152 - 260`, which would place the object at the very bottom of the screen. To add more padding between the object and the bottom screen subtract more pixels as the third term (e.g. `object_y_pos = 1152 - 260 - 65`). Note, you can either use integers or predefined strings. See the doc for further information:
https://zulko.github.io/moviepy/getting_started/compositing.html#positioning-clips

#### About Video Codec
Codec to use for image encoding. Can be any codec supported by ffmpeg. If the filename is has extension ‘.mp4’, ‘.ogv’, ‘.webm’, the codec will be set accordingly, but you can still set it if you don’t like the default. For other extensions, the output filename must be set accordingly.
https://zulko.github.io/moviepy/ref/VideoClip/VideoClip.html#moviepy.video.VideoClip.VideoClip.write_videofile

#### About Bitrate (CBR) and ffmpeg_params (VBR)
You can either use the moviepy’s `bitrate` property or you can use `ffmpeg_params` (using qscale) to control the quality of the video. More information about CBR/VBR:
https://trac.ffmpeg.org/wiki/Encode/MPEG-4

#### Parameter Description


| Parameter                | Description|
|:-------------------------|:-----------|
| `object_x_pos`           | X Position |
| `object_y_pos`           | Y Position |
| `folder_file_delimiter`  | The character that should be used within the final filename to denote a file’s parent folder. For example, `Folder1/File1.avi` gets `Folder1-File1.avi` |
| `object_extension`       | Set your object file extension here. This is used safely strip away the file extension when merging folder and file names. |
| `video_extension`        | Set your desired output extension here. |
| `use_codec`              | See description above|
| `use_bitrate`            | CBR bitrate. Expecting a string ("10M")|
| `use_ffmpeg_params`      | ffmpeg parameters. Expecting key-value-pairs (e.g., ffmpeg `-qscale:v 2` gets: `["-qscale:v", "2"]`)|
| `use_preset`             | Sets the time that FFMPEG will spend optimizing the compression (only affects mp4/H.264). Choices are: `ultrafast, superfast, veryfast, faster, fast, medium (default), slow, slower, veryslow`. Note that this does not impact the quality of the video, only the size of the video file. So choose ultrafast when you are in a hurry and file size does not matter.|
| `use_sound`              | Toggles to render video with sound (True/False)|
| `output_folder`          | Name of the folder in which all videos files should be exported|
| `write_filelist`         | Toggle if the script should write a filelist.txt, which updates after each render iteration. That means, after the render process is completed, the filelist.txt reflects the videos that were rendered in the out folder.|
| `use_composition_queue`  | If set to `True`, the script will only generate the video/object combinations listed in the queue.txt. That means, you can create an explicit paring line-by-line. If set to `False`, all video/object combinations from within the folders will be generated.|

In [ ]:
object_x_pos = "center"
object_y_pos = 1152 - 260 - 65
folder_file_delimter = "_"
object_extension = "png"
video_extension = "wmv" # wmv, avi
use_codec = "msmpeg4" # msmpeg4 (for wmv), libxvid (xvid for avi)
use_bitrate = None # CBR: "10M" # use this for constant
use_ffmpeg_params = ["-qscale", "2"] # VBR: ["-qscale", "2"]
use_preset = "ultrafast" # "ultrafast", default is medium
use_sound = False
output_folder = "out"
write_filelist = True
use_composition_queue = False

### 📝 Generate File List & Queue



In [ ]:
# Check if a queue file is actually there, if not make sure use_composition_queue is False
if not Path("queue.txt").is_file():
    use_composition_queue = False

# Use Composition Queue List if True
if use_composition_queue:
    with open("queue.txt") as inp:
          video_object_couples = list(zip(*(line.strip().split('\t') for line in inp)))

    video_file_list = video_object_couples[0]
    object_file_list = video_object_couples[1]

else: # Else create a video and object file list containing all videos and objects combinations
    video_file_list = []
    object_file_list = []
    # (1) video file list
    for current_video_folder in video_folders:
        current_video_files = list(Path(current_video_folder).glob("**/*"))
        for current_video_file in current_video_files:
            video_file_list.append(str(current_video_file))

    # (2) object file list
    for current_object_folder in object_folders:
        current_object_files = list(Path(current_object_folder).glob("**/*"))
        for current_object_file in current_object_files:
            object_file_list.append(str(current_object_file))


# Delete Composition Queue List, only if use_composition_queue is False
if not use_composition_queue:
    queue = open("queue.txt", "w")
    queue.close()

    # Generate a full composition queue list, only if use_composition_queue is False
    for current_vid in video_file_list:
        for current_obj in object_file_list:
            # write composition queue
            with open("queue.txt", "a") as queue:
                queue.write(current_vid + "\t" + current_obj + "\n")

# Check if video output folder is there, if not create it
Path("./" + output_folder).mkdir(parents=True, exist_ok=True)

### 🔁 Process Loop

In [ ]:
# start counting
start_time = time.time()

# Clean filelist if flag is set
if write_filelist:
    filelist = open("filelist.txt", "w")
    filelist.close()


# line-by-line paring (if use_composition_queue is True)
if use_composition_queue:
    video_counter = 0
    print("Start Processing Videos")
    for i in range(len(video_file_list)):

        video_counter += 1

        print(f"\nVideo {video_counter} of {len(video_file_list)}...")
        # compose current file name
        current_file_name = video_file_list[i]\
                            .replace("/", folder_file_delimter)\
                            .replace("\\", folder_file_delimter)\
                            .replace("." + video_extension, folder_file_delimter) +\
                            object_file_list[i]\
                            .replace("/", folder_file_delimter)\
                            .replace("\\", folder_file_delimter)\
                            .replace("." + object_extension, "." + video_extension)

        # write filelist if enabled
        if write_filelist:
            with open("filelist.txt", "a") as filelist:
                filelist.write(current_file_name + "\n")
        
        # Process Videos
        mp_obj1 = mp.VideoFileClip(video_file_list[i]).set_duration(11)
        mp_obj2 = ( 
            mp.ImageClip(object_file_list[i])
            .set_duration(mp_obj1.duration)
            .set_pos((object_x_pos, object_y_pos))
        )

        mp_composition = mp.CompositeVideoClip([mp_obj1, mp_obj2])
        mp_composition.write_videofile("./" + output_folder + "/" + current_file_name, \
            codec=use_codec, bitrate=use_bitrate, ffmpeg_params=use_ffmpeg_params, audio=use_sound, preset=use_preset)
        # Closing the clip avoids Errno 12 (Cannot allocate memory)
        mp_composition.close()
# End of Line-by-Line Loop



# Video Loop for all combinations (use_composition_queue is False)
if not use_composition_queue:
    video_counter = 0
    print("Start Processing Videos")
    for current_vid in video_file_list:
        for current_obj in object_file_list: # for i,current_obj in enumerate(object_file_list):

            video_counter += 1

            print(f"\nVideo {video_counter} of {len(video_file_list) * len(object_file_list)}...")
            # compose current file name
            current_file_name = current_vid\
                                .replace("/", folder_file_delimter)\
                                .replace("\\", folder_file_delimter)\
                                .replace("." + video_extension, folder_file_delimter) +\
                                current_obj\
                                .replace("/", folder_file_delimter)\
                                .replace("\\", folder_file_delimter)\
                                .replace("." + object_extension, "." + video_extension)

            # write filelist if enabled
            if write_filelist:
                with open("filelist.txt", "a") as filelist:
                    filelist.write(current_file_name + "\n")
            
            # Process Videos
            mp_obj1 = mp.VideoFileClip(current_vid).set_duration(11)
            mp_obj2 = ( 
                mp.ImageClip(current_obj)
                .set_duration(mp_obj1.duration)
                .set_pos((object_x_pos, object_y_pos))
            )

            mp_composition = mp.CompositeVideoClip([mp_obj1, mp_obj2])
            mp_composition.write_videofile("./" + output_folder + "/" + current_file_name, \
            codec=use_codec, bitrate=use_bitrate, ffmpeg_params=use_ffmpeg_params, audio=use_sound, preset=use_preset)
            # Closing the clip avoids Errno 12 (Cannot allocate memory)
            mp_composition.close()
# End of Video Loop


print("💯 all done")
print(f"Execution time in seconds: {time.time() - start_time:.2f}")


## ▶ Re-Order Video Files Script
### TODO